In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import requests
from bs4 import BeautifulSoup
from time import sleep
import random
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

% matplotlib inline

In [2]:
with open("flight_list_20.txt") as f:
    fl = [i.strip('"') for i in f.read().split(',')]
print(len(fl))

6083


In [3]:
breakpoint = fl.index('SWA1530')
print(breakpoint)
print(fl[breakpoint])

1302
SWA1530


In [4]:
flight_ls = fl[(breakpoint+1):]

In [5]:
def parse_times(Dep_or_Arr):
    
    try:
        aG = Dep_or_Arr[Dep_or_Arr.index('Actual')+1].strip()
    except:
        aG = ' '
    
    try:
        aTaxi = Dep_or_Arr[Dep_or_Arr.index('Actual')+2].strip()
    except:
        aTaxi = ' '
        
    aTakeoff = Dep_or_Arr[Dep_or_Arr.index('Actual')+3].strip()

    sG = Dep_or_Arr[Dep_or_Arr.index('Scheduled')+1].strip()
    sTakeoff = Dep_or_Arr[Dep_or_Arr.index('Scheduled')+2].strip()
    
    try:
        Dep_or_Arr.index('Average Delay')
        delay = ' '.join(Dep_or_Arr[Dep_or_Arr.index('Average Delay')+1].strip().split())
    except:
        delay = ' '

    return aG, aTaxi, aTakeoff, sG, sTakeoff, delay
    
    
def write_detail (info):
#     print(info)
    try:
        Dep = info[info.index('DEPARTURE TIMES'):info.index('ARRIVAL TIMES')]
        Arr = info[info.index('ARRIVAL TIMES'):-2]
    
        Dep_info = list(parse_times(Dep))
        Arr_info = list(parse_times(Arr))

        return Dep_info + Arr_info
    except:
        simple = [' ',' ',info[7],' ',info[10],' ',' ',' ',info[8],' ',info[11],' ']
        return simple

def write_space (info_s):
#     print(info_s)
    for x in info_s:
        if ('Speed Filed:' in x):
            speed = x[len('Speed Filed:')+1:]

        if ('Altitude Filed:' in x):
            altitute = x[len('Altitude Filed:')+1:]

        if ('Actual:' in x):
            a_Dis = x[len('Actual:')+1: x.find('(Planned:')]

        if ('Actual:' in x):
            p_Dis = x[x.find('(Planned:')+len('(Planned:'): x.find('/Direct:')]

        if ('Route' in x):
            route = x[7:]
    ls = []
    try:
        ls.append(speed)
    except:
        ls.append(' ')
    try:
        ls.append(altitute)
    except:
        ls.append(' ')
    try:
        ls.append(a_Dis)
    except:
        ls.append(' ')
    try:
        ls.append(p_Dis)
    except:
        ls.append(' ')
    try:
        ls.append(route)
    except:
        ls.append(' ')
#     print(ls)
    return ls

In [6]:
def expand_histories (history_tb, flight_id):
    ls_history = []
    for link in history_tb.find_all('a')[1:]:
        if 'history' in link['href']:
            if len(link['href']) > 45:
                ls_history.append(link['href'])
    
    dict1 = defaultdict(list)
    
    for link in ls_history:

        flight_date = link[link.find('history/')+8:link.find('history/')+16]
        print(link)
        if int(flight_date) > 20170929:
            continue
        print(flight_date)
        flight_airports = [link[-8:-5],link[-3:]]
        if flight_airports[0] not in airport_list:
            continue
        if flight_airports[1] not in airport_list:
            continue 

        url = 'https://flightaware.com{}'
        path = url.format(link)
        
        
#         try:
#             chromedriver = "/Applications/chromedriver"
#             os.environ["webdriver.chrome.driver"] = chromedriver
#             browser=webdriver.Chrome(chromedriver)
#             browser.set_page_load_timeout(10)
#             browser.get(path)
#         except:
#             print('no connection to site')
#             continue

        chromedriver = "/Applications/chromedriver"
        os.environ["webdriver.chrome.driver"] = chromedriver
        browser=webdriver.Chrome(chromedriver)
#         browser.set_page_load_timeout(10)
        browser.get(path)

    
        try:
            element1 = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="slideOutPanel"]/div[1]/div[2]/div[4]/div[5]'))
            )
        except:
            print("can't get element1")
            continue
        info = element1.text.split('\n')
        
        try:
            element2 = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="slideOutPanel"]/div[1]/div[2]/div[4]/div[7]'))
            )
        except:
            print("can't get element2")
            continue
        
        info_s = element2.text.split('\n')
        
        a = write_detail(info)
        b = write_space(info_s)
        
        sleep(.1+.2*random.random())
        browser.close()
        
        # get input row in dictionary format
        # key = col_name
        dict1[flight_id].append([flight_date] + flight_airports + a + b) 
    try:
        final.update(dict1)
    except:
        pass

In [7]:
%%time

from collections import defaultdict
columns=(['ID','ORI','DES','Act_Gate_Dep','Taxi'
          ,'Act_Takeoff','Sch_Dep','Sch_Takeoff'
         ,'AvgDepDelay','Act_Landing_Arr','Taxi_Arr'
            ,'Act_Takeoff','Sch_Landing_Arr','Sch_Gate_Arr'
         ,'AvgArrDelay','Speed','Altitute','Act_Dist','Sch_Dist','Route'])

airport_list = (['ATL', 'ORD', 'LAX', 'DFW', 'JFK', 'DEN', 'SFO'
                 , 'LAS', 'CLT', 'MIA', 'PHX', 'IAH', 'SEA', 'MCO'
                 , 'EWR', 'MSP', 'BOS', 'DTW', 'PHL', 'LGA'])

final = {}

for flight_id in flight_ls:
    url = 'https://flightaware.com/live/flight/{}/history'
    path = url.format(flight_id)
    response = requests.get(path)
    
    print(path, response.status_code)
    
    if response.status_code == 200:
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        history_tb = soup.find_all('table')[2]
        
        expand_histories(history_tb, flight_id)
    else:
        continue
print(final)

https://flightaware.com/live/flight/AAL1658/history 200
/live/flight/AAL1658/history/20171004/2241Z/KDFW/KCLT
/live/flight/AAL1658/history/20171003/2241Z/KDFW/KCLT
/live/flight/AAL1658/history/20171002/2241Z/KDFW/KCLT
/live/flight/AAL1658/history/20171001/2241Z/KDFW/KCLT
/live/flight/AAL1658/history/20170929/2241Z/KDFW/KCLT
20170929
/live/flight/AAL1658/history/20170928/2241Z/KDFW/KCLT
20170928
/live/flight/AAL1658/history/20170927/2241Z/KDFW/KCLT
20170927
/live/flight/AAL1658/history/20170926/2241Z/KDFW/KCLT
20170926
/live/flight/AAL1658/history/20170925/2241Z/KDFW/KCLT
20170925
/live/flight/AAL1658/history/20170924/2241Z/KDFW/KCLT
20170924
/live/flight/AAL1658/history/20170922/2241Z/KDFW/KCLT
20170922
/live/flight/AAL1658/history/20170921/2241Z/KDFW/KCLT
20170921
/live/flight/AAL1658/history/20170920/2241Z/KDFW/KCLT
20170920
/live/flight/AAL1658/history/20170919/2241Z/KDFW/KCLT
20170919
https://flightaware.com/live/flight/UAL1968/history 200
/live/flight/UAL1968/history/20171004/1925

https://flightaware.com/live/flight/VRD1899/history 200
/live/flight/VRD1899/history/20171004/1851Z/KLAX/KSFO
/live/flight/VRD1899/history/20171003/1851Z/KLAX/KSFO
/live/flight/VRD1899/history/20171002/1851Z/KLAX/KSFO
/live/flight/VRD1899/history/20171001/1851Z/KLAX/KSFO
/live/flight/VRD1899/history/20170930/1851Z/KLAX/KSFO
/live/flight/VRD1899/history/20170929/1851Z/KLAX/KSFO
20170929
/live/flight/VRD1899/history/20170928/1851Z/KLAX/KSFO
20170928
/live/flight/VRD1899/history/20170927/1851Z/KLAX/KSFO
20170927
/live/flight/VRD1899/history/20170926/1851Z/KLAX/KSFO
20170926
/live/flight/VRD1899/history/20170925/1851Z/KLAX/KSFO
20170925
/live/flight/VRD1899/history/20170924/1851Z/KLAX/KSFO
20170924
/live/flight/VRD1899/history/20170923/1851Z/KLAX/KSFO
20170923
/live/flight/VRD1899/history/20170922/1851Z/KLAX/KSFO
20170922
/live/flight/VRD1899/history/20170921/1851Z/KLAX/KSFO
20170921
/live/flight/VRD1899/history/20170920/1851Z/KLAX/KSFO
20170920
/live/flight/VRD1899/history/20170919/1851Z/

/live/flight/SWA186/history/20170928/1045Z/KMDW/KPHX
20170928
/live/flight/SWA186/history/20170928/0400Z/KLAS/KLAX
20170928
/live/flight/SWA186/history/20170927/2350Z/KSTL/KLAS
20170927
/live/flight/SWA186/history/20170927/2140Z/KDAL/KSTL
20170927
/live/flight/SWA186/history/20170927/1735Z/KSFO/KDAL
20170927
/live/flight/SWA186/history/20170927/1500Z/KPHX/KSFO
20170927
/live/flight/SWA186/history/20170927/1045Z/KMDW/KPHX
20170927
/live/flight/SWA186/history/20170927/0400Z/KLAS/KLAX
20170927
/live/flight/SWA186/history/20170926/2350Z/KSTL/KLAS
20170926
/live/flight/SWA186/history/20170926/2140Z/KDAL/KSTL
20170926
/live/flight/SWA186/history/20170926/1735Z/KSFO/KDAL
20170926
/live/flight/SWA186/history/20170926/1500Z/KPHX/KSFO
20170926
/live/flight/SWA186/history/20170926/1045Z/KMDW/KPHX
20170926
/live/flight/SWA186/history/20170926/0400Z/KLAS/KLAX
20170926
/live/flight/SWA186/history/20170925/2350Z/KSTL/KLAS
20170925
/live/flight/SWA186/history/20170925/2140Z/KDAL/KSTL
20170925
/live/fl

/live/flight/RPA5898/history/20170928/1010Z/KLGA/KDFW
20170928
/live/flight/RPA5898/history/20170927/1010Z/KLGA/KDFW
20170927
/live/flight/RPA5898/history/20170926/1010Z/KLGA/KDFW
20170926
/live/flight/RPA5898/history/20170925/1010Z/KLGA/KDFW
20170925
/live/flight/RPA5898/history/20170924/1200Z/KRSW/KLGA
20170924
/live/flight/RPA5898/history/20170922/1010Z/KLGA/KDFW
20170922
/live/flight/RPA5898/history/20170921/1010Z/KLGA/KDFW
20170921
/live/flight/RPA5898/history/20170920/1010Z/KLGA/KDFW
20170920
/live/flight/RPA5898/history/20170919/1010Z/KLGA/KDFW
20170919
https://flightaware.com/live/flight/UAL2047/history 200
/live/flight/UAL2047/history/20171004/2320Z/KBOS/KEWR
/live/flight/UAL2047/history/20171004/1750Z/KIAH/KBOS
/live/flight/UAL2047/history/20171003/2320Z/KBOS/KEWR
/live/flight/UAL2047/history/20171003/1750Z/KIAH/KBOS
/live/flight/UAL2047/history/20171002/2320Z/KBOS/KEWR
/live/flight/UAL2047/history/20171002/1750Z/KIAH/KBOS
/live/flight/UAL2047/history/20171001/2320Z/KBOS/KEWR

https://flightaware.com/live/flight/AAL431/history 200
/live/flight/AAL431/history/20171004/2355Z/KDFW/KLAS
/live/flight/AAL431/history/20171004/1934Z/KLAX/KDFW
/live/flight/AAL431/history/20171003/2355Z/KDFW/KLAS
/live/flight/AAL431/history/20171003/1934Z/KLAX/KDFW
/live/flight/AAL431/history/20171002/2355Z/KDFW/KLAS
/live/flight/AAL431/history/20171002/1934Z/KLAX/KDFW
/live/flight/AAL431/history/20171001/2355Z/KDFW/KLAS
/live/flight/AAL431/history/20171001/1934Z/KLAX/KDFW
/live/flight/AAL431/history/20170930/2355Z/KDFW/KLAS
/live/flight/AAL431/history/20170930/1934Z/KLAX/KDFW
/live/flight/AAL431/history/20170929/2355Z/KDFW/KLAS
20170929
/live/flight/AAL431/history/20170929/1934Z/KLAX/KDFW
20170929
/live/flight/AAL431/history/20170928/2355Z/KDFW/KLAS
20170928
/live/flight/AAL431/history/20170928/1934Z/KLAX/KDFW
20170928
/live/flight/AAL431/history/20170927/2355Z/KDFW/KLAS
20170927
/live/flight/AAL431/history/20170927/1934Z/KLAX/KDFW
20170927
/live/flight/AAL431/history/20170926/2355Z/

...

/live/flight/AAL252R/history/20170920/2302Z/KDFW/KSFO
20170920
/live/flight/AAL252R/history/20170919/2150Z/KDFW/KSFO
20170919
https://flightaware.com/live/flight/DAL2343/history 200
/live/flight/DAL2343/history/20171004/1450Z/KEWR/KATL
/live/flight/DAL2343/history/20171004/1159Z/KATL/KEWR
/live/flight/DAL2343/history/20171003/1450Z/KEWR/KATL
/live/flight/DAL2343/history/20171003/1159Z/KATL/KEWR
/live/flight/DAL2343/history/20171002/1450Z/KEWR/KATL
/live/flight/DAL2343/history/20171002/1159Z/KATL/KEWR
/live/flight/DAL2343/history/20171001/1450Z/KEWR/KATL
/live/flight/DAL2343/history/20170929/1500Z/KEWR/KATL
20170929
/live/flight/DAL2343/history/20170929/1159Z/KATL/KEWR
20170929
/live/flight/DAL2343/history/20170928/1500Z/KEWR/KATL
20170928
/live/flight/DAL2343/history/20170928/1159Z/KATL/KEWR
20170928
/live/flight/DAL2343/history/20170927/1500Z/KEWR/KATL
20170927
/live/flight/DAL2343/history/20170927/1159Z/KATL/KEWR
20170927
/live/flight/DAL2343/history/20170926/1500Z/KEWR/KATL
20170926

/live/flight/DAL1460/history/20170927/1000Z/KMIA/KATL
20170927
/live/flight/DAL1460/history/20170926/1000Z/KMIA/KATL
20170926
/live/flight/DAL1460/history/20170925/1000Z/KMIA/KATL
20170925
/live/flight/DAL1460/history/20170923/1000Z/KMIA/KATL
20170923
/live/flight/DAL1460/history/20170922/1000Z/KMIA/KATL
20170922
/live/flight/DAL1460/history/20170921/1000Z/KMIA/KATL
20170921
/live/flight/DAL1460/history/20170920/1000Z/KMIA/KATL
20170920
/live/flight/DAL1460/history/20170919/1000Z/KMIA/KATL
20170919
https://flightaware.com/live/flight/DAL1894/history 200
/live/flight/DAL1894/history/20171004/2020Z/KCLT/KMSP
/live/flight/DAL1894/history/20171003/2020Z/KCLT/KMSP
/live/flight/DAL1894/history/20171002/2020Z/KCLT/KMSP
/live/flight/DAL1894/history/20171001/2020Z/KCLT/KMSP
/live/flight/DAL1894/history/20170930/1825Z/KSFO/KCVG
/live/flight/DAL1894/history/20170930/1255Z/KCVG/KSFO
/live/flight/DAL1894/history/20170929/1830Z/KSFO/KCVG
20170929
/live/flight/DAL1894/history/20170929/1300Z/KCVG/KSFO

/live/flight/DAL8772/history/20170930/1450Z/KATL/KMSP
/live/flight/DAL8772/history/20170923/1110Z/KDTW/KTPA
20170923
https://flightaware.com/live/flight/RPA3724/history 200
/live/flight/RPA3724/history/20171004/2345Z/KOKC/KIAH
/live/flight/RPA3724/history/20171003/2345Z/KOKC/KIAH
/live/flight/RPA3724/history/20171002/2345Z/KOKC/KIAH
/live/flight/RPA3724/history/20171001/2345Z/KOKC/KIAH
/live/flight/RPA3724/history/20170930/1444Z/KATL/KEWR
/live/flight/RPA3724/history/20170929/2345Z/KOKC/KIAH
20170929
/live/flight/RPA3724/history/20170928/2345Z/KOKC/KIAH
20170928
/live/flight/RPA3724/history/20170927/2345Z/KOKC/KIAH
20170927
/live/flight/RPA3724/history/20170926/2345Z/KOKC/KIAH
20170926
/live/flight/RPA3724/history/20170925/2345Z/KOKC/KIAH
20170925
/live/flight/RPA3724/history/20170924/2345Z/KOKC/KIAH
20170924
/live/flight/RPA3724/history/20170923/1444Z/KATL/KEWR
20170923
/live/flight/RPA3724/history/20170922/2345Z/KOKC/KIAH
20170922
/live/flight/RPA3724/history/20170921/2345Z/KOKC/KIAH

/live/flight/AAL1555/history/20170928/1345Z/KMCO/KJFK
20170928
/live/flight/AAL1555/history/20170927/1345Z/KMCO/KJFK
20170927
/live/flight/AAL1555/history/20170926/1345Z/KMCO/KJFK
20170926
/live/flight/AAL1555/history/20170925/1345Z/KMCO/KJFK
20170925
/live/flight/AAL1555/history/20170924/1345Z/KMCO/KJFK
20170924
/live/flight/AAL1555/history/20170923/1055Z/KMCO/KJFK
20170923
/live/flight/AAL1555/history/20170922/1345Z/KMCO/KJFK
20170922
/live/flight/AAL1555/history/20170921/1345Z/KMCO/KJFK
20170921
/live/flight/AAL1555/history/20170920/1345Z/KMCO/KJFK
20170920
/live/flight/AAL1555/history/20170919/1345Z/KMCO/KJFK
20170919
https://flightaware.com/live/flight/AAL1696/history 200
/live/flight/AAL1696/history/20171004/1940Z/KDFW/KSEA
/live/flight/AAL1696/history/20171003/1940Z/KDFW/KSEA
/live/flight/AAL1696/history/20171002/1940Z/KDFW/KSEA
/live/flight/AAL1696/history/20171002/0045Z/KSEA/KDFW
/live/flight/AAL1696/history/20171001/1940Z/KDFW/KSEA
/live/flight/AAL1696/history/20170930/1940Z/

KeyboardInterrupt: 

for 5 lines, CPU times: user 2.42 s, sys: 197 ms, total: 2.62 s
Wall time: 4min 46s

In [8]:
final

{'AAL1': [['20170929',
   'JFK',
   'LAX',
   '08:49AM EDT',
   '23 minutes',
   '09:12AM EDT',
   '08:00AM EDT',
   '08:00AM EDT',
   ' ',
   '11:54AM PDT',
   '26 minutes',
   '12:20PM PDT',
   '10:46AM PDT',
   '11:22AM PDT',
   ' ',
   '542 mph',
   '34,000 ft',
   '2,571 mi ',
   ' 2,537 mi',
   '211N/09218W DBL CHESZ Q88 PROMT Q88 HAKMN ANJLL1'],
  ['20170928',
   'JFK',
   'LAX',
   '07:54AM EDT',
   '21 minutes',
   '08:15AM EDT',
   '08:00AM EDT',
   '08:00AM EDT',
   ' ',
   '10:52AM PDT',
   '26 minutes',
   '11:18AM PDT',
   '11:14AM PDT',
   '11:22AM PDT',
   ' ',
   '541 mph',
   '30,000 ft',
   '2,501 mi ',
   ' 2,503 mi',
   'BV J230 AIR J80 BAYLI J80 MCI J24 SLN J102 ALS J44 RSK J64 TBC JASSE Q90 DNERO ANJLL1'],
  ['20170927',
   'JFK',
   'LAX',
   '08:02AM EDT',
   '22 minutes',
   '08:24AM EDT',
   '08:00AM EDT',
   '08:00AM EDT',
   ' ',
   '11:01AM PDT',
   '12 minutes',
   '11:13AM PDT',
   '11:12AM PDT',
   '11:22AM PDT',
   ' ',
   '531 mph',
   '30,000 ft',
  

In [9]:
test = (pd.DataFrame.from_dict(final, orient = 'index')
       .stack()
       .reset_index(level=1, drop=True)
       .reset_index())

In [10]:
test

,index,0
0,AAL1658,"[20170929, DFW, CLT, 05:39PM CDT, 14 minutes, ..."
1,AAL1658,"[20170928, DFW, CLT, 08:48PM CDT, 26 minutes, ..."
2,AAL1658,"[20170927, DFW, CLT, 05:59PM CDT, 14 minutes, ..."
3,AAL1658,"[20170926, DFW, CLT, 05:37PM CDT, 13 minutes, ..."
4,AAL1658,"[20170925, DFW, CLT, 05:46PM CDT, 14 minutes, ..."
5,AAL1658,"[20170924, DFW, CLT, 06:41PM CDT, 23 minutes, ..."
6,AAL1658,"[20170922, DFW, CLT, 05:51PM CDT, 10 minutes, ..."
7,AAL1658,"[20170921, DFW, CLT, 05:37PM CDT, 10 minutes, ..."
8,AAL1658,"[20170920, DFW, CLT, 06:20PM CDT, 12 minutes, ..."
9,AAL1658,"[20170919, DFW, CLT, 05:38PM CDT, 13 minutes, ..."


In [11]:
test.to_csv('flight_details10.csv')

In [12]:
test.tail()

,index,0
1969,AAL336,"[20170923, ORD, LGA, 07:46PM CDT, 21 minutes, ..."
1970,AAL336,"[20170922, ORD, LGA, 07:27PM CDT, 23 minutes, ..."
1971,AAL336,"[20170921, ORD, LGA, 07:26PM CDT, 29 minutes, ..."
1972,AAL336,"[20170920, ORD, LGA, 07:24PM CDT, 17 minutes, ..."
1973,AAL336,"[20170919, ORD, LGA, 07:51PM CDT, 22 minutes, ..."


In [13]:
test.iloc[2,1]

['20170927',
 'DFW',
 'CLT',
 '05:59PM CDT',
 '14 minutes',
 '06:13PM CDT',
 '05:41PM CDT',
 '05:41PM CDT',
 ' ',
 '09:23PM EDT',
 '6 minutes',
 '09:29PM EDT',
 '08:58PM EDT',
 '09:15PM EDT',
 ' ',
 '526 mph',
 '37,000 ft',
 '1,018 mi ',
 ' 997 mi',
 'ACHH2 BSKAT LIT BNA TAZZA FILPZ3']